# Setup

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport

%matplotlib inline

In [ ]:
import astropy
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import re 
from astropy.table import Table
import astropy.table
import json
from scipy import stats
from copy import deepcopy

from relaxed import halo_parameters, halo_catalogs, analysis
import warnings

# Fits to obtain alpha

With Phil, I already discussed how adding $\beta$ to the fitting procedure does not contribute much because it has a strong degeneracy with $\alpha$. So in the following notebook we only use $\alpha$ for the fit. 

**Notes:**

 - There are two data points that don't reach a_{1/2} ever, this explains the runtime error below when obtaining a_{1/2}

In [ ]:
from relaxed.progenitors.catalog import get_alpha, lma_fit
from relaxed.analysis import setup, get_ma

In [ ]:
hcat, indices, scales = setup('m11')
zs = (1/ scales) -1 
ma = get_ma(hcat.cat, indices)
lma = np.log(ma)[:, :160] # towards very big scales many masses become really close to 0. 

# remove infs 
keep = np.zeros(len(hcat.cat))
for indx in range(len(lma)):
    if np.isinf(lma[indx]).sum() == 0 and np.isnan(lma[indx]).sum() == 0: 
        keep[indx] = 1

keep = keep.astype(bool)
hcat.cat = hcat.cat[keep]
ma = ma[keep]
lma = lma[keep]
n = lma.shape[1]
zs = zs[:n]
print(len(hcat.cat))
print(lma.shape)
print(zs.shape)

In [ ]:
# calculate all alpha, betas and add them to table. 
alphas = [] 
for idx in range(len(hcat.cat)):
    alpha = get_alpha(zs, lma[idx])
    alphas.append(alpha)
c1 = astropy.table.Column(alphas, name='alpha')
hcat.cat.add_column(c1)

In [ ]:
# add a1/2 
from relaxed.halo_parameters import A2
a2 = A2.from_cat(hcat.cat, scales, indices)
c = astropy.table.Column(a2, name='a2')
hcat.cat.add_column(c)

In [ ]:
from relaxed import plots, plot_funcs
from relaxed.halo_parameters import get_hparam
from collections import OrderedDict

# round up params.
params = [
    "mvir",
    "eta",
    "x0",
    "v0",
    "q",
    "cvir",
    "phi_l",
]
hparams = [get_hparam(param, log=True) for param in params]
hparams.append(get_hparam("alpha", log=False))
hparams.append(get_hparam("f_sub", log=False))
hparams.append(get_hparam("a2", log=False))
# hparams.append(get_hparam("chi2", log=False))
hparams.append(get_hparam("gamma_tdyn", log=False))

params.append('alpha')
params.append('f_sub')
params.append('a2')
# params.append('chi2')
params.append('gamma_tdyn')

plot_func = plot_funcs.MatrixValues(xlabel_size=24, ylabel_size=24)
plot = plots.MatrixPlot(
    plot_func, hparams, nrows=1, ncols=1, figsize=(10, 10), figpath='../../figures/'
)

# load catalogs
plot.load(hcat)

names = ['Bolshoi']
plot_params = OrderedDict({param: {*names} for param in params})
plot.generate(plot_params)

fname = 'matrix3.pdf'
plot.save(fname=fname)